In [1]:
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv('data/train.txt', skipinitialspace=True)
df_train.rename(columns={'Timestamp (DHHMM)':'time'}, inplace=True)
df_train.keys()

Index([u'time', u'S1', u'S2', u'S3', u'S4', u'S5', u'S6', u'S7', u'S8', u'S9',
       u'S10', u'S11', u'S12', u'S13', u'S14', u'S15', u'S16', u'S17', u'S18',
       u'S19', u'S20', u'S21', u'S22', u'S23', u'S24', u'S25', u'S26', u'S27',
       u'S28', u'S29', u'S30', u'S31', u'S32', u'S33', u'S34', u'S35', u'S36',
       u'S37', u'S38', u'S39', u'S40', u'S41', u'S42', u'S43', u'S44', u'S45',
       u'S46', u'S47', u'S48', u'S49', u'S50', u'S51', u'S52', u'S53', u'S54',
       u'S55', u'S56'],
      dtype='object')

In [3]:
df_train.sort(u'time')
df_train.head()

/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,time,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S47,S48,S49,S50,S51,S52,S53,S54,S55,S56
0,10000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10001,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,10002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# converts time stamp as int in DHHMM into index for the table
def timestamptoindex(ts):
    mm = ts % 100
    hh = (ts % (100 * 100)) // 100
    d = ts // (100 * 100)
    
    index = mm + 60 * hh + 60 * 24 * (d - 1)
    return index

In [5]:
timestamptoindex(10000)

0

In [6]:
# no exception means, functions works alright!
for i, row in df_train.iterrows():
    assert timestamptoindex(row[u'time']) == i

## 1. Basic Linear model for one sensor only

In [7]:
df_train.keys()

Index([u'time', u'S1', u'S2', u'S3', u'S4', u'S5', u'S6', u'S7', u'S8', u'S9',
       u'S10', u'S11', u'S12', u'S13', u'S14', u'S15', u'S16', u'S17', u'S18',
       u'S19', u'S20', u'S21', u'S22', u'S23', u'S24', u'S25', u'S26', u'S27',
       u'S28', u'S29', u'S30', u'S31', u'S32', u'S33', u'S34', u'S35', u'S36',
       u'S37', u'S38', u'S39', u'S40', u'S41', u'S42', u'S43', u'S44', u'S45',
       u'S46', u'S47', u'S48', u'S49', u'S50', u'S51', u'S52', u'S53', u'S54',
       u'S55', u'S56'],
      dtype='object')

In [8]:
# want to predict for the missing, use the valid ones as trainset
missing_percentage = 1. * df_train_missing.count()[0] / df_train_valid.count()[0]
print missing_percentage
print str(df_train_missing.count()[0]) + '/' + str(df_train_valid.count()[0]) + ' missing'

NameError: name 'df_train_missing' is not defined

In [9]:
# sklearn functions used for the linear regression model
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_absolute_error

In [10]:
_ = plt.hist(df_train['S1'])

In [11]:
df_train['S1'].unique()

array([ 0,  2,  1,  5,  3,  4,  6,  7,  9, 14, 10,  8, 17, 15, 20, 30, 16,
       23, 18, 19, -1, 11, 13, 26, 12])

In [21]:
df_train

,time,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S48,S49,S50,S51,S52,S53,S54,S55,S56,btime
0,10000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10001,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,10002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,10005,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
6,10006,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
7,10007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
8,10008,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
9,10009,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [16]:
## function to compute results for a basic multinomial, logistic model
def MultinomialTimeModel(df):
    dfout = df.copy()
    
    # extract time feature
    #df['btime'] = (df['time'] % (100 * 100)) // 100
    # extract time feature (5minute features!)
    df['btime'] = (df['time'] % (100 * 100)) // 100 * 100 + ((df['time'] % 100) // 5) * 5
    
    # encode btime as categorical features
    encoder = OneHotEncoder(sparse=True) 
    categoricalFeat = df['btime']
    categoricals_encoded = encoder.fit_transform(categoricalFeat.reshape(-1, 1))

    # for each sensor, build a logistic regression model based on the time
    num_sensors = 56
    for i in xrange(num_sensors):
        sid = 'S' + str(i+1)
        print 'predicting ' + sid + ' missing data...'

        idx_missing = df[sid] < 0

        X_train = categoricals_encoded.toarray()[np.array(~idx_missing), :]
        X_test = categoricals_encoded.toarray()[np.array(idx_missing), :]
        y_train, y_test = df[sid][~idx_missing], df[sid][idx_missing]

        # check if errors exist, if not skip next steps!
        if y_test.shape[0] == 0:
            print sid + ' has no errors!'
            continue
        
        clf = sklearn.linear_model.LinearRegression()#LogisticRegression(multi_class='multinomial', solver='newton-cg')

        # fit the model!
        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)

        # update the entries according to the model
        dfout[sid] = 0
        dfout[sid][idx_missing] = y_pred
    print 'done!'
    return dfout

In [17]:
# OUPUT RESULTS TO FILE
dfout = MultinomialTimeModel(df_train)

predicting S1 missing data...
predicting S2 missing data...
predicting S3 missing data...
predicting S4 missing data...
predicting S5 missing data...
predicting S6 missing data...
predicting S7 missing data...
predicting S8 missing data...
predicting S9 missing data...
predicting S10 missing data...
predicting S11 missing data...
predicting S12 missing data...
predicting S13 missing data...
predicting S14 missing data...
predicting S15 missing data...
predicting S16 missing data...
predicting S17 missing data...
predicting S18 missing data...
predicting S19 missing data...
predicting S20 missing data...
predicting S21 missing data...
predicting S22 missing data...
predicting S23 missing data...
predicting S24 missing data...
predicting S25 missing data...
predicting S26 missing data...
predicting S27 missing data...
predicting S28 missing data...
predicting S29 missing data...
predicting S30 missing data...
predicting S31 missing data...
predicting S32 missing data...
predicting S33 mi

/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
from helper import *

In [20]:
create_submission_file(dfout, 'multilog.csv')

## 2. Data transformation

In [22]:
# encode data new
df = load_train_data()

In [23]:
rows

,time,S1,S2,S3,S4,S5,S6,S7,S8,S9,...,S47,S48,S49,S50,S51,S52,S53,S54,S55,S56
0,10000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10001,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,10002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
